In [0]:
### This code analyzes fire incidents in San Francisco
### Steps
### Get CSV
### CSV File Size and number of rows
  ### 2 GB and 5 M de registros
### Create table and rename columns
### Repartition (Qty) and byPeriod?
  ###How to know the best number of partitions
    ### 16 (CSV) x 9 (Parquet)
### Convert to Parquet
### Load Parquet
### Compare Load time CSV x Parquet group by por year(date)
  ### 50s (CSV) x 16s (Parquet)
  ### Use SparkSQL
### Exploratory Analysis
### Use Spark SQL - Group By + Sort Operations
### Test ML Library?
### Test Graphiz?
### Test Spark Streaming?
### View temporaria x View Permanente

In [0]:
###Next Commit 2
#### Title:
      ### Analysis CSV(Python), Parquet(Python) and Table(SQL)  in terms of Performance
        ##### Time
        ##### Number of Partitions

In [0]:
#Import libs
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, TimestampType #Used for schema
from pyspark.sql.functions import * # in order to use Date Functions - Ex.: extract Year from Date
#from pyspark.sql.functions import to_timestamp

In [0]:
schema = StructType([
StructField("CallNumber",StringType(),True),
StructField("UnitID",StringType(),True),
StructField("IncidentNumber",StringType(),True),
StructField("CallType",StringType(),True),
StructField("CallDate",StringType(),True),
StructField("WatchDate",StringType(),True),
StructField("ReceivedDtTm",StringType(),True),
StructField("EntryDtTm",StringType(),True),
StructField("DispatchDtTm",StringType(),True),
StructField("ResponseDtTm",StringType(),True),
StructField("OnSceneDtTm",StringType(),True),
StructField("TransportDtTm",StringType(),True),
StructField("HospitalDtTm",StringType(),True),
StructField("CallFinalDisposition",StringType(),True),
StructField("AvailableDtTm",StringType(),True),
StructField("Address",StringType(),True),
StructField("City",StringType(),True),
StructField("ZipcodeofIncident",StringType(),True),
StructField("Battalion",StringType(),True),
StructField("StationArea",StringType(),True),
StructField("Box",StringType(),True),
StructField("OriginalPriority",StringType(),True),
StructField("Priority",StringType(),True),
StructField("FinalPriority",StringType(),True),
StructField("ALSUnit",StringType(),True),
StructField("CallTypeGroup",StringType(),True),
StructField("NumberofAlarms",StringType(),True),
StructField("UnitType",StringType(),True),
StructField("Unitsequenceincalldispatch",StringType(),True),
StructField("FirePreventionDistrict",StringType(),True),
StructField("SupervisorDistrict",StringType(),True),
StructField("Neighborhooods-AnalysisBoundaries",StringType(),True),
StructField("RowID",StringType(),True),
StructField("case_location",StringType(),True),
StructField("AnalysisNeighborhoods",StringType(),True)
  ])

In [0]:
#Get the file on this link
#https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3
  
# File location and type
file_location = "/FileStore/tables/Fire_Department_Calls_for_Service.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(schema) \
  .load(file_location)


In [0]:
df.count()

Out[153]: 5516263

In [0]:
display(df.limit(3))

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,ReceivedDtTm,EntryDtTm,DispatchDtTm,ResponseDtTm,OnSceneDtTm,TransportDtTm,HospitalDtTm,CallFinalDisposition,AvailableDtTm,Address,City,ZipcodeofIncident,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumberofAlarms,UnitType,Unitsequenceincalldispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhooods-AnalysisBoundaries,RowID,case_location,AnalysisNeighborhoods
210690030,T03,21030278,Alarms,03/10/2021,03/09/2021,03/10/2021 12:16:03 AM,03/10/2021 12:18:36 AM,03/10/2021 12:19:01 AM,03/10/2021 12:20:15 AM,null,null,null,Fire,03/10/2021 12:28:42 AM,1300 Block of MARKET ST,San Francisco,94102,B02,36,3111,3,3,3,false,Alarm,1,TRUCK,3,2,6,Tenderloin,210690030-T03,POINT (-122.41697932641094 37.7770834641944),36
210391607,E19,21017645,Alarms,02/08/2021,02/08/2021,02/08/2021 01:00:14 PM,02/08/2021 01:01:36 PM,02/08/2021 01:01:40 PM,02/08/2021 01:03:21 PM,02/08/2021 01:05:44 PM,null,null,Fire,02/08/2021 01:18:09 PM,400 Block of SERRANO DR,San Francisco,94132,B08,19,8581,3,3,3,true,Alarm,1,ENGINE,1,8,7,Lakeshore,210391607-E19,POINT (-122.48045074945836 37.7190118676788),16
210391164,T04,21017596,Alarms,02/08/2021,02/08/2021,02/08/2021 10:54:56 AM,02/08/2021 10:56:50 AM,02/08/2021 10:56:57 AM,02/08/2021 10:57:07 AM,02/08/2021 10:59:34 AM,null,null,Fire,02/08/2021 11:06:42 AM,600 Block of LONG BRIDGE ST,San Francisco,94158,B03,04,2264,3,3,3,false,Alarm,1,TRUCK,1,3,6,Mission Bay,210391164-T04,POINT (-122.39227179213904 37.77288298280324),4


In [0]:
df.printSchema()

root
-- CallNumber: string (nullable = true)
-- UnitID: string (nullable = true)
-- IncidentNumber: string (nullable = true)
-- CallType: string (nullable = true)
-- CallDate: string (nullable = true)
-- WatchDate: string (nullable = true)
-- ReceivedDtTm: string (nullable = true)
-- EntryDtTm: string (nullable = true)
-- DispatchDtTm: string (nullable = true)
-- ResponseDtTm: string (nullable = true)
-- OnSceneDtTm: string (nullable = true)
-- TransportDtTm: string (nullable = true)
-- HospitalDtTm: string (nullable = true)
-- CallFinalDisposition: string (nullable = true)
-- AvailableDtTm: string (nullable = true)
-- Address: string (nullable = true)
-- City: string (nullable = true)
-- ZipcodeofIncident: string (nullable = true)
-- Battalion: string (nullable = true)
-- StationArea: string (nullable = true)
-- Box: string (nullable = true)
-- OriginalPriority: string (nullable = true)
-- Priority: string (nullable = true)
-- FinalPriority: string (nullable = true)
-- ALSUnit: string (nullable = true)
-- CallTypeGroup: string (nullable = true)
-- NumberofAlarms: string (nullable = true)
-- UnitType: string (nullable = true)
-- Unitsequenceincalldispatch: string (nullable = true)
-- FirePreventionDistrict: string (nullable = true)
-- SupervisorDistrict: string (nullable = true)
-- Neighborhooods-AnalysisBoundaries: string (nullable = true)
-- RowID: string (nullable = true)
-- case_location: string (nullable = true)
-- AnalysisNeighborhoods: string (nullable = true)

In [0]:
from_pattern = "MM/dd/yyyy"
to_pattern = "yyyy-MM-dd"

In [0]:
df = df.withColumn("CallDateTs",to_timestamp(df["CallDate"], from_pattern))
display(df.limit(3))

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,ReceivedDtTm,EntryDtTm,DispatchDtTm,ResponseDtTm,OnSceneDtTm,TransportDtTm,HospitalDtTm,CallFinalDisposition,AvailableDtTm,Address,City,ZipcodeofIncident,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumberofAlarms,UnitType,Unitsequenceincalldispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhooods-AnalysisBoundaries,RowID,case_location,AnalysisNeighborhoods,CallDateTs
210690030,T03,21030278,Alarms,03/10/2021,03/09/2021,03/10/2021 12:16:03 AM,03/10/2021 12:18:36 AM,03/10/2021 12:19:01 AM,03/10/2021 12:20:15 AM,null,null,null,Fire,03/10/2021 12:28:42 AM,1300 Block of MARKET ST,San Francisco,94102,B02,36,3111,3,3,3,false,Alarm,1,TRUCK,3,2,6,Tenderloin,210690030-T03,POINT (-122.41697932641094 37.7770834641944),36,2021-03-10T00:00:00.000+0000
210391607,E19,21017645,Alarms,02/08/2021,02/08/2021,02/08/2021 01:00:14 PM,02/08/2021 01:01:36 PM,02/08/2021 01:01:40 PM,02/08/2021 01:03:21 PM,02/08/2021 01:05:44 PM,null,null,Fire,02/08/2021 01:18:09 PM,400 Block of SERRANO DR,San Francisco,94132,B08,19,8581,3,3,3,true,Alarm,1,ENGINE,1,8,7,Lakeshore,210391607-E19,POINT (-122.48045074945836 37.7190118676788),16,2021-02-08T00:00:00.000+0000
210391164,T04,21017596,Alarms,02/08/2021,02/08/2021,02/08/2021 10:54:56 AM,02/08/2021 10:56:50 AM,02/08/2021 10:56:57 AM,02/08/2021 10:57:07 AM,02/08/2021 10:59:34 AM,null,null,Fire,02/08/2021 11:06:42 AM,600 Block of LONG BRIDGE ST,San Francisco,94158,B03,04,2264,3,3,3,false,Alarm,1,TRUCK,1,3,6,Mission Bay,210391164-T04,POINT (-122.39227179213904 37.77288298280324),4,2021-02-08T00:00:00.000+0000


In [0]:
df.groupby(year("CallDateTs")).count().show()

+----------------+------+
year(CallDateTs)| count|
+----------------+------+
 2003|240457|
 2007|235856|
 2018|310700|
 2015|296244|
 2006|235440|
 2013|273097|
 2014|280905|
 2019|319855|
 2004|235353|
 2020|285743|
 2012|265596|
 2009|244633|
 2016|303967|
 2001|220328|
 2005|232919|
 2000|158154|
 2010|256174|
 2011|268689|
 2008|249690|
 2017|312471|
+----------------+------+
only showing top 20 rows

In [0]:
df = df.withColumn("CallYear",year(df["CallDateTs"]))
display(df.limit(3))

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,ReceivedDtTm,EntryDtTm,DispatchDtTm,ResponseDtTm,OnSceneDtTm,TransportDtTm,HospitalDtTm,CallFinalDisposition,AvailableDtTm,Address,City,ZipcodeofIncident,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumberofAlarms,UnitType,Unitsequenceincalldispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhooods-AnalysisBoundaries,RowID,case_location,AnalysisNeighborhoods,CallDateTs,CallYear
210690030,T03,21030278,Alarms,03/10/2021,03/09/2021,03/10/2021 12:16:03 AM,03/10/2021 12:18:36 AM,03/10/2021 12:19:01 AM,03/10/2021 12:20:15 AM,null,null,null,Fire,03/10/2021 12:28:42 AM,1300 Block of MARKET ST,San Francisco,94102,B02,36,3111,3,3,3,false,Alarm,1,TRUCK,3,2,6,Tenderloin,210690030-T03,POINT (-122.41697932641094 37.7770834641944),36,2021-03-10T00:00:00.000+0000,2021
210391607,E19,21017645,Alarms,02/08/2021,02/08/2021,02/08/2021 01:00:14 PM,02/08/2021 01:01:36 PM,02/08/2021 01:01:40 PM,02/08/2021 01:03:21 PM,02/08/2021 01:05:44 PM,null,null,Fire,02/08/2021 01:18:09 PM,400 Block of SERRANO DR,San Francisco,94132,B08,19,8581,3,3,3,true,Alarm,1,ENGINE,1,8,7,Lakeshore,210391607-E19,POINT (-122.48045074945836 37.7190118676788),16,2021-02-08T00:00:00.000+0000,2021
210391164,T04,21017596,Alarms,02/08/2021,02/08/2021,02/08/2021 10:54:56 AM,02/08/2021 10:56:50 AM,02/08/2021 10:56:57 AM,02/08/2021 10:57:07 AM,02/08/2021 10:59:34 AM,null,null,Fire,02/08/2021 11:06:42 AM,600 Block of LONG BRIDGE ST,San Francisco,94158,B03,04,2264,3,3,3,false,Alarm,1,TRUCK,1,3,6,Mission Bay,210391164-T04,POINT (-122.39227179213904 37.77288298280324),4,2021-02-08T00:00:00.000+0000,2021


In [0]:
resultPath = "/user/df_fire"
df.write.mode("overwrite").format("parquet").partitionBy("CallYear").parquet(resultPath)

In [0]:

# File location and type
file_location_parquet = "/user/df_fire"
file_type = "parquet"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

df_parquet = spark.read.format(file_type) \
  .load(file_location_parquet)

In [0]:
df_parquet.groupby(year("CallDateTs")).count().show()

+----------------+------+
year(CallDateTs)| count|
+----------------+------+
 2018|310700|
 2013|273097|
 2019|319855|
 2012|265596|
 2004|235353|
 2020|285743|
 2009|244633|
 2017|312471|
 2007|235856|
 2006|235440|
 2014|280905|
 2001|220328|
 2010|256174|
 2003|240457|
 2015|296244|
 2016|303967|
 2011|268689|
 2008|249690|
 2002|225951|
 2005|232919|
+----------------+------+
only showing top 20 rows

In [0]:
df.rdd.getNumPartitions()

Out[147]: 16

In [0]:
df_parquet.rdd.getNumPartitions()

Out[148]: 9

In [0]:
#Create a table across any clusters
permanent_table_name = 'df_fire'
df_parquet.write.format("parquet").mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select
  CallYear,
  count(*) as Qty
from
  df_fire
group by
  CallYear

CallYear,Qty
2018,310700
2013,273097
2019,319855
2012,265596
2003,240457
2015,296244
2016,303967
2011,268689
2008,249690
2002,225951


In [0]:
%sql
select
  CallType,
  count(*) as Qty
from
  df_fire
where CallYear = 2021
group by
  CallType
  order by Qty desc

CallType,Qty
Medical Incident,43065
Alarms,7938
Structure Fire,4995
Traffic Collision,2376
Outside Fire,1254
Other,1219
Citizen Assist / Service Call,929
Water Rescue,602
Gas Leak (Natural and LP Gases),451
Electrical Hazard,413
